In [ ]:
!pip install llama-index
!pip install slack_sdk
!pip install qdrant_client

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv
from llama_index import GPTListIndex, SlackReader

import logging
import structlog
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

load_dotenv()
load_dotenv(dotenv_path=find_dotenv(filename=".env.local"), override=True)

In [ ]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
logging.getLogger("urllib3.connectionpool").setLevel(logging.CRITICAL)
logging.getLogger("urllib3.util.retry").setLevel(logging.CRITICAL)
logging.getLogger("openai").setLevel(logging.CRITICAL)
logging.getLogger("httpcore").setLevel(logging.CRITICAL)

In [ ]:
import json
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

embedding_model = "text-embedding-ada-002"

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(10))
def create_embedding_with_backoff(**kwargs):
    return openai.Embedding.create(**kwargs)

def create_embedding(message, num: int | None, total: int | None):
    logger.debug("Creating embedding", num=num, total=total)
    embedding = create_embedding_with_backoff(
        input=json.dumps(message),
        model=embedding_model,
    )["data"][0]["embedding"]
    return {
        "embedding": embedding,
        "message": message,
    }

In [ ]:
from collections import namedtuple
from queue import Queue

from attr import dataclass
from notion_client import Client
from notion_client.helpers import collect_paginated_api

NotionBlockResult = namedtuple(
    "NotionBlockResult",
    ["content_components", "sub_pages"],
)

@dataclass
class NotionBlockContent:
    contents: str
    inline: bool

@dataclass
class NotionBlockIdentifier:
    identifier: str

NotionBlockContentOrIdentifier = NotionBlockContent | NotionBlockIdentifier

def traverse_notion_block(
    notion_client: Client,
    task_queue: Queue,
    block_id,
) -> NotionBlockResult:
    print(f"Getting children blocks of {block_id}")

    block_children = collect_paginated_api(
        notion_client.blocks.children.list,
        block_id=block_id,
    )

    sub_pages = []
    content_components: list[NotionBlockContentOrIdentifier] = []

    for block_child in block_children:
        result_obj = block_child[block_child["type"]]

        is_subpage = block_child["type"] == "child_page"
        
        if "rich_text" in result_obj:
            for rich_text in result_obj["rich_text"]:
                # skip if doesn't have text object
                if "text" in rich_text:
                    text = rich_text["text"]["content"]
                    content_components.append(NotionBlockContent(contents=text, inline=True))
                elif "plain_text" in rich_text:
                    text = rich_text["plain_text"]
                    content_components.append(NotionBlockContent(contents=text, inline=True))
        elif "title" in result_obj:
            text = result_obj["title"]
            if is_subpage:
                content_components.append(NotionBlockContent(contents=text, inline=False))
                # Kind of hacky but otherwise we don't get a newline after the subpage
                content_components.append(NotionBlockContent(contents="", inline=False))
            else:
                content_components.append(NotionBlockContent(contents=text, inline=True))
            
        if block_child["has_children"]:
            task_queue.put(block_child["id"])
            if is_subpage:
                sub_pages.append(block_child["id"])
            else:
                content_components.append(NotionBlockIdentifier(block_child["id"]))
        else:
            # Kind of hacky but otherwise we're not getting newlines between text blocks
            content_components.append(NotionBlockContent(contents="", inline=False))

    return NotionBlockResult(content_components=content_components, sub_pages=sub_pages)

In [ ]:
import concurrent.futures
import os
import queue
import threading

from notion_client import Client

integration_token = os.getenv("NOTION_API_KEY")
page_ids = ["52b92f24e14b43be83f64c206f211413"]

notion_client = Client(auth=integration_token)

executor = concurrent.futures.ThreadPoolExecutor()
task_queue = queue.Queue()
future_queue = queue.Queue()
results = {}


def process_task_queue():
    while True:
        block_id = task_queue.get()
        future = executor.submit(
            traverse_notion_block,
            notion_client,
            task_queue,
            block_id,
        )
        results[block_id] = future
        future_queue.put(future)
        task_queue.task_done()

# Hacky but if we start a second thread to ensure that all the futures are complete, we avoid race conditions that occur 
# from the interactions of executor.submit and task_queue.task_done
def process_future_queue():
    while True:
        future = future_queue.get()
        future.result()
        future_queue.task_done()

task_queue_thread = threading.Thread(target=process_task_queue)
task_queue_thread.start()
future_queue_thread = threading.Thread(target=process_future_queue)
future_queue_thread.start()


for page_id in page_ids:
    task_queue.put(page_id)

task_queue.join()
future_queue.join()

In [ ]:
results

In [ ]:
pages_to_explore = [*page_ids]
explored_pages = set()
pages_to_contents = {}


def collapse_text(root_block_id, depth):
    block_result = results[root_block_id].result()
    pages_to_explore.extend(block_result.sub_pages)

    indent = (depth * "\t")
    block_contents = [indent]
    
    for content_component in block_result.content_components:
        if isinstance(content_component, NotionBlockIdentifier):
            block_contents.append("\n" + indent)
            block_contents.append(collapse_text(content_component.identifier, depth + 1))
        else:
            if not content_component.inline:
                block_contents.append("\n" + indent)
            
            block_contents.append(content_component.contents)
    
    return "".join(block_contents)

while len(pages_to_explore) > 0:
    page_id = pages_to_explore.pop(0)
    if page_id in explored_pages:
        continue
    explored_pages.add(page_id)

    print(f"Collapsing text for page {page_id}")
    pages_to_contents[page_id] = collapse_text(page_id, 0)

In [ ]:
pages_to_contents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30,
    length_function=len,
)

documents = []

for page_id, contents in pages_to_contents.items():
    chunks = text_splitter.split_text(contents)
    for chunk_index, chunk in enumerate(chunks):
        documents.append({"text": chunk, "metadata": {"parent_page_id": page_id, "chunk_idx": chunk_index}})

In [ ]:
def create_payload(document):
    return {
        "text": document.text if document.text else None,
        "doc_id": document.doc_id if document.doc_id else None,
        "extra_info": document.extra_info if document.extra_info else None
    }

In [ ]:
embedding = create_embedding(message=create_payload(documents[0]), num=0, total=1)

In [ ]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor() as executor:
    embeddings = list(
        executor.map(
            lambda idx_item: create_embedding(
                message=create_payload(idx_item[1]),
                num=idx_item[0],
                total=len(documents),
            ),
            enumerate(documents),
        ),
    )

In [ ]:
from llama_index import LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))
prompt_helper = PromptHelper.from_llm_predictor(llm_predictor=llm_predictor)
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    chunk_size_limit=1024,
    prompt_helper=prompt_helper,
)

In [ ]:
from llama_index import GPTQdrantIndex
import qdrant_client

client = qdrant_client.QdrantClient(
    url="http://localhost:6333"
)
index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("My docker-compose isn't working, how can I fix it?")
response

In [ ]:
response.response